In [1]:
import numpy as np
np.set_printoptions(formatter={'float': '{: 0.5f}'.format}, suppress = True)

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib notebook

### File reading

In [2]:
# Reads the measurement data and returns an array of measurement locations and measured data
def read_mfile(filename):
    Z = []
    m = []
    
    f = open(filename, "r")
    
    for line in f:
        l = line.replace("\n", "").split(' ')
        for i in range(len(l)):
            l[i] = float(l[i])
        Z.append(l[0])
        m.append([l[0], l[1], l[2], l[3], l[4], l[5]])
    
    Z = np.array(Z)
    for i in range(len(m)):
        m[i] = np.array(m[i])
    
    return (Z, m)

# Reads a BField file and returns a dictionary containing a set of vectors representing the field
def read_Bfile(filename, Z):
    B   = {}
    for z in Z:
        B[z] = [[], [], [], [], [], []]

    f = open(filename, "r")

    for line in f:
        l = line.replace("\n", "").split(' ')
        for i in range(len(l)):
            l[i] = float(l[i])
        for i in range(len(l)):
            B[l[2]][i].append(l[i])

    for z in B:
        for i in range(len(B[z])):
            B[z][i] = np.array(B[z][i])
    
    return B

### BField functions

In [3]:
# Gets the approximated magnetic field at a point using the nearest neighbour criteria
def get_B(x0, y0, z0, Z, B):
    for zi in range(len(Z)):
        if zi+1 == len(Z):
            z0 = Z[zi]
        elif (z0 > Z[zi] and z0 < Z[zi+1]):
            z0 = Z[zi]
    if z0 in B:
        b  = B[z0]
        
        d_x = np.abs(b[0] - x0)
        d_y = np.abs(b[1] - y0)
        
        x_min = d_x.min()
        y_min = d_y.min()
        
        Xi = np.where(d_x == x_min)
        Yi = np.where(d_y == y_min)
        
        for xi in Xi[0]:
            for yi in Yi[0]:
                if xi == yi:
                    return (b[3][xi], b[4][xi], b[5][xi])
        return (None, None, None)
    else:
        return (None, None, None)

# Plots the magnetic field B at a particular measurement plane z
def plot_B(z, B):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.quiver(B[z][0], B[z][1], B[z][2], B[z][3], B[z][4], B[z][5], length=0.1, normalize=False)

    plt.show()

### Transport

In [4]:
# default step size
step_size = 0.2

def get_A(tx, ty, bx, by, bz):
    c = np.sqrt(1 + tx*tx + ty*ty)
    return [c * ( ty * (tx*bx + bz) - (1 + tx*tx) * by), c * (-tx * (ty*by + bz) + (1 + ty*ty) * bx)]

def get_dA(tx, ty, bx, by, bz):
    c2 = 1 + tx*tx + ty*ty
    c  = np.sqrt(c2)
    Ax = c * ( ty * (tx*bx + bz) - (1 + tx*tx) * by)
    Ay = c * (-tx * (ty*by + bz) + (1 + ty*ty) * bx)
    
    dA = []
    dA.append(tx*Ax/c2 + c * ( ty*bx - 2*tx*by)) # dAx/dtx
    dA.append(ty*Ax/c2 + c * ( tx*bx + bz))      # dAx/dty
    dA.append(tx*Ay/c2 + c * (-ty*by - bz))      # dAy/dtx
    dA.append(ty*Ay/c2 + c * (-tx*by + 2*ty*bx)) # dAy/dty
    
    return dA

def k_transport(i, f, ivec, icov, v, Z, B, prnt, realB): # returns fvec and fcov
    # temporary state vector to improve legibility
    z  = ivec[0]
    x  = ivec[1]
    y  = ivec[2]
    tx = ivec[3]
    ty = ivec[4]
    Q  = ivec[5]

    fcov = np.copy(icov)

    # step size computation
    n_steps = (int) (abs((Z[i] - Z[f]) / step_size) + 1)
    s = np.sign(Z[f] - Z[i]) * step_size
    
    for j in range(n_steps):
        if j == n_steps-1:
            s = np.sign(Z[f] - Z[i]) * abs(z - Z[f])
        
        if realB: (Bx, By, Bz) = get_B(x, y, z, Z, B)
        else:     (Bx, By, Bz) = (1., 1., 0.)
        
        A  = get_A(tx, ty, Bx, By, Bz)
        dA = get_dA(tx, ty, Bx, By, Bz)
        
        # propagate the covariance matrix
        dx_dtx  = s
        dy_dtx  = 0.5*Q*v * s*s * dA[2]
        dty_dtx = Q*v * s * dA[2]
        
        dx_dty  = 0.5*Q*v * s*s * dA[1]
        dy_dty  = s;
        dtx_dty = Q*v * s * dA[1]
        
        dx_dQ   = 0.5*v * s*s * A[0]
        dtx_dQ  = v * s * A[0]
        dy_dQ   = 0.5*v * s*s * A[1]
        dty_dQ  = v * s * A[1];
        
        # Jacobian matrix
        F = np.array(
            [[1, 0,  dx_dtx,  dx_dty,  dx_dQ],
             [0, 1,  dy_dtx,  dy_dty,  dy_dQ],
             [0, 0,       1, dtx_dty, dtx_dQ],
             [0, 0, dty_dtx,       1, dty_dQ],
             [0, 0,       0,       0,      1]]
        )
        
        # C = FCF^T
        fcov = np.dot(np.dot(F, fcov), F.transpose())
        
        # Q process noise matrix estimate
        p = abs(1./Q)
        pz = p / np.sqrt(1 + tx*tx + ty*ty)
        px = tx * pz
        py = ty * pz
        
        t_ov_X0 = np.sign(Z[f] - Z[i]) * s / 14 # ArgonRadLen = 14
        mass = 0.000510998
        if Q > 0:
            mass = 0.938272029
        beta = p*p / np.sqrt(p*p + mass*mass)
        cos_angle = abs((x*px + y*py + z*pz) / np.sqrt(x*x + y*y + z*z) * p)
        path_len = t_ov_X0 / cos_angle
        
        sct_RMS = (0.0136 / (beta)) * np.sqrt(path_len) * (1 + 0.038*np.log(path_len)) # Highland-Lynch-Dahl formula
        
        cov_txtx = (1 + tx*tx) * (1 + tx*tx + ty*ty) * sct_RMS*sct_RMS
        cov_txty = (1 + ty*ty) * (1 + tx*tx + ty*ty) * sct_RMS*sct_RMS
        cov_tyty = tx*ty * (1 + tx*tx + ty*ty) * sct_RMS*sct_RMS
        
        if s > 0:
            fcov[2,2] += cov_txtx
            fcov[2,3] += cov_txty
            fcov[3,2] += cov_txty
            fcov[3,3] += cov_tyty
        
        # propagate the state vector
        z += s
        x += tx * s + 0.5 * Q*v * A[0] * s*s
        y += ty * s + 0.5 * Q*v * A[1] * s*s
        tx += Q * v * A[0] * s
        ty += Q * v * A[1] * s
        
    fvec = np.array([z, x, y, tx, ty, Q])
    if prnt >= 2:
        print("  transport:  ", fvec)
    if prnt >= 3:
        print("    matrix %2d:\n" % f, fcov)
    return (fvec, fcov)

### Filter

In [5]:
def getH(y, s, w, l):
    h1 = -np.tan(np.radians(6*s))
    return [1, h1 - w * (4/l) * (1 - y/(l/2))]

def geth(x, y, s, w, l):
    return x - np.tan(np.radians(6*s)) * y + w*(1 - y/(l/2)) * (1 - y/(l/2))

def k_filter(k, ivec, icov, chi2, prnt): # returns fvec, fcov and chi2
    # filter the covariance matrix
    if abs(np.linalg.det(icov)) < 1.e-30:
        print("error, initial covariance matrix is singular!")
        return
    
    K = np.zeros(5)
    V = abs(m[k][2])
    H = getH(ivec[2], m[k][3], m[k][4], m[k][5])
    
    # Sherman-Morrison formula
    Hv   = np.array([[H[0]], [H[1]], [0], [0], [0]])
    HvT  = np.transpose(Hv)
    
    div = V + np.dot(np.dot(HvT, icov), Hv)[0, 0]
    res = np.dot(np.dot(np.dot(icov, Hv), HvT), icov)
    res = np.true_divide(res, div)
    
    fcov = np.subtract(icov, res)
    if abs(np.linalg.det(fcov)) < 1.e-30:
        print("error, final covariance matrix is singular!")
        return

    # filter the state vector
    for j in range(5):
        K[j] = (H[0] * fcov[j,0] + H[1] * fcov[j,1])/V
        
    h = geth(ivec[1], ivec[2], m[k][3], m[k][4], m[k][5])
    chi2 += (m[k][1] - h) * (m[k][1] - h) / V
    
    fvec = np.copy(ivec)
    for j in range(5):
        fvec[j+1] += K[j] * (m[k][1] - h)
    
    if prnt >= 2:
        print("  filter:     ", fvec)
    if prnt >= 3:
        print("    matrix f:\n", fcov)
        print("    chi2: %3f\n" % chi2)
        
    return (fvec, fcov, chi2)

### Kalman Filter

In [6]:
def kfitter(I, s, S, Z, B, prnt, realB):
    K = len(Z) # total number of measurements
    v = 0.002997924580 # speed of light
    
    # best last state vector, covariance matrix and fit's chi^2 fit probability found
    sK_b = np.zeros(6)
    SK_b = np.zeros((5,5))
    b_chi2 = np.Inf
    
    # run the Kalman Filter
    for i in range(I):
        c_chi2 = 0
        if i > 0:
            (s[0], S[0]) = k_transport(K-1, 0, s[K-1], S[K-1], v, Z, B, prnt, realB)
        for k in range(1, K):
            (s[k], S[k]) = k_transport(k-1, k, s[k-1], S[k-1], v, Z, B, prnt, realB)
            (s[k], S[k], c_chi2) = k_filter(k, s[k], S[k], c_chi2, prnt)
        if np.isnan(c_chi2): # NOTE: IDK why this is happening
            break
        if c_chi2 >= b_chi2:
            print("iteration %2d : no improvement" % i)
            continue
        if prnt >= 1:
            print("iteration %2d :" % i, s[K-1], " (%.2f)" % c_chi2)
        if  abs(s[K-1][5] - sK_b[5]) < 5.e-4 and \
            abs(s[K-1][1] - sK_b[1]) < 1.e-4 and \
            abs(s[K-1][2] - sK_b[2]) < 1.e-4 and \
            abs(s[K-1][3] - sK_b[3]) < 1.e-6 and \
            abs(s[K-1][4] - sK_b[4]) < 1.e-6:
            i = I
        sK_b = s[K-1]
        SK_b = S[K-1]
        b_chi2 = c_chi2

    if i == 1:
        sK_b = s[len(s)-1]
        SK_b = S[len(s)-1]
    
    return (sK_b, SK_b)

### Get data from files

In [7]:
(Z, m) = read_mfile("m.txt")
B = read_Bfile("B.txt", Z)

FileNotFoundError: [Errno 2] No such file or directory: 'm.txt'

### KFitter

In [8]:
# config
prnt = 1 # how much info to print (from 0 to 3)
I = 30 # number of iterations

# state vector and covariance matrix arrays
s = []
S = []

for i in range(len(Z)):
    s.append(np.zeros(6))
    S.append(np.zeros((5,5)))

# initial state vector and covariance matrix
s[0] = np.array([Z[0], 12.834542, 18.629339,  0.048048,  0.102449, -0.588641])
S[0] = np.array(
    [[27.809679,  0.941768, -0.109315, -0.004673,  0.009938],
     [ 0.941768, 85.895073, -0.002793, -0.410291,  0.003917],
     [-0.109315, -0.002793,  0.000481,  0.000020, -0.000130],
     [-0.004673, -0.410291,  0.000020,  0.002811, -0.000046],
     [ 0.009938,  0.003917, -0.000130, -0.000046,  0.000733]]
)

(sK_b, SK_b) = kfitter(I, s, S, Z, B, prnt, False)

iteration  0 : [ 528.96033  2.70545  32.46450 -0.04286 -0.00747 -0.08863]  (1747.47)
iteration  1 : [ 528.96033  1.96956  37.93620 -0.07959  0.09901  0.07392]  (426.74)
iteration  2 : [ 528.96033  1.28847  41.24033 -0.10247  0.13327  0.12725]  (105.07)
iteration  3 : [ 528.96033  0.84156  43.02022 -0.11592  0.15145  0.15655]  (57.73)
iteration  4 : [ 528.96033  0.52888  44.12206 -0.12475  0.16276  0.17516]  (37.52)
iteration  5 : [ 528.96033  0.30745  44.87957 -0.13093  0.17055  0.18804]  (26.87)
iteration  6 : [ 528.96033  0.14907  45.44067 -0.13548  0.17629  0.19750]  (20.53)
iteration  7 : [ 528.96033  0.03255  45.87693 -0.13896  0.18071  0.20473]  (16.43)
iteration  8 : [ 528.96033 -0.05642  46.22770 -0.14172  0.18423  0.21044]  (13.61)
iteration  9 : [ 528.96033 -0.12675  46.51730 -0.14395  0.18712  0.21506]  (11.59)
iteration 10 : [ 528.96033 -0.18398  46.76210 -0.14580  0.18955  0.21888]  (10.10)
iteration 11 : [ 528.96033 -0.23161  46.97373 -0.14735  0.19163  0.22209]  (8.97)
i